In [2]:
import numpy as np
import pandas as pd
import unicodedata
import re

from bs4 import BeautifulSoup
from markdown import markdown

In [3]:
ruta_data = "../data"
ruta_train = ruta_data + "/train.csv"
ruta_test = ruta_data + "/test.csv"

In [4]:
def markdown_to_text(s):
    return s.replace("*", " ")

In [7]:
"""def markdown_to_text(markdown_string):
    ''' Converts a markdown string to plaintext '''

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text"""

'def markdown_to_text(markdown_string):\n    \'\'\' Converts a markdown string to plaintext \'\'\'\n\n    # md -> html -> text since BeautifulSoup can extract text cleanly\n    html = markdown(markdown_string)\n\n    # remove code snippets\n    html = re.sub(r\'<pre>(.*?)</pre>\', \' \', html)\n    html = re.sub(r\'<code>(.*?)</code >\', \' \', html)\n\n    # extract text\n    soup = BeautifulSoup(html, "html.parser")\n    text = \'\'.join(soup.findAll(text=True))\n\n    return text'

In [8]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [9]:
train_origi = pd.read_csv(ruta_train, usecols=["descripcion"])
test_origi = pd.read_csv(ruta_test, usecols=["descripcion"])

In [10]:
train_origi = train_origi.fillna("")
test_origi = test_origi.fillna("")

### Veamos que forma tienen las descripciones actuales:

In [11]:
train_origi.loc[349].descripcion

'vendo bonita casa en fracc sobre forjadores, caseta de vigilancia.\r\nsala, comedor, cocina  con area para desayunador, bodega, patio trasero, medio baño, sala de t.v. 3 recamaras amplias,principal con vestidor y baño, baño compartido. cochera 2 autos.\r\ncitas:   '

In [12]:
train_origi.loc[302].descripcion

'casa con 20 años de edad en buenas condiciones,espacios amplios, iluminada, construida en 3 niveles, 5 habitaciones amplias, 3 baños completos, no cuenta con estacionamiento, cuarto de servicio, cuarto de lavado, cocina integral, mosaico en cocina y baños, pisos de loseta española, 2 tinacos, cercana a escuelas, hospitales, iglesia san juan bautista, comercios, metro bus tenayuca, universidades, avenida vallejo, colinda con d.f. terreno de 123 m2 construcción de 245 m2. se acepta credito bancario, infonavit, fovissste'

In [13]:
train_origi.loc[802].descripcion

'<p><strong>venta de casa **queretaro** el mirador</strong></p>\r\n<p>casa de 104.5 metros de construccion en el mirador queretaro**, hermosa casa **nueva**&nbsp; tres recamaras 2 1/2 ba&ntilde;os***sport club **con alberca semiolimpica**gimnasio*canchas de futboll 7**tennis** ludoteca*kids club****video vigilancia las 24 hrs del dia por internet y accesos controlados**.terreno 90 mtsplanta baja**estancia,sala de tv,ba&ntilde;o de visitas (1/2 ba&ntilde;o),comedor,cocina,patio de servicio,2 cajon de estacionamiento,jardin posterior.planta alta**recamara principal con ba&ntilde;o completo vestidor c/balcon,**segunda recamaras c closet, ba&ntilde;o completo..preparacion jacussi en roof garden**</p>\r\n<p>&nbsp;</p>'

In [14]:
train_origi["descripcion"] = train_origi.apply(lambda x: BeautifulSoup(x["descripcion"], "lxml").text, axis=1)
test_origi["descripcion"] = test_origi.apply(lambda x: BeautifulSoup(x["descripcion"], "lxml").text, axis=1)

/home/julian/.local/lib/python3.7/site-packages/bs4/__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/julian/.local/lib/python3.7/site-packages/bs4/__init__.py:272: UserWarning: "b'/'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


### Veamos como quedaron las descripciones ahora:

In [15]:
train_origi.loc[349].descripcion

'vendo bonita casa en fracc sobre forjadores, caseta de vigilancia.\r\nsala, comedor, cocina  con area para desayunador, bodega, patio trasero, medio baño, sala de t.v. 3 recamaras amplias,principal con vestidor y baño, baño compartido. cochera 2 autos.\r\ncitas:   '

In [16]:
train_origi.loc[302].descripcion

'casa con 20 años de edad en buenas condiciones,espacios amplios, iluminada, construida en 3 niveles, 5 habitaciones amplias, 3 baños completos, no cuenta con estacionamiento, cuarto de servicio, cuarto de lavado, cocina integral, mosaico en cocina y baños, pisos de loseta española, 2 tinacos, cercana a escuelas, hospitales, iglesia san juan bautista, comercios, metro bus tenayuca, universidades, avenida vallejo, colinda con d.f. terreno de 123 m2 construcción de 245 m2. se acepta credito bancario, infonavit, fovissste'

In [17]:
train_origi.loc[802].descripcion

'venta de casa **queretaro** el mirador\ncasa de 104.5 metros de construccion en el mirador queretaro**, hermosa casa **nueva**\xa0 tres recamaras 2 1/2 baños***sport club **con alberca semiolimpica**gimnasio*canchas de futboll 7**tennis** ludoteca*kids club****video vigilancia las 24 hrs del dia por internet y accesos controlados**.terreno 90 mtsplanta baja**estancia,sala de tv,baño de visitas (1/2 baño),comedor,cocina,patio de servicio,2 cajon de estacionamiento,jardin posterior.planta alta**recamara principal con baño completo vestidor c/balcon,**segunda recamaras c closet, baño completo..preparacion jacussi en roof garden**\n\xa0'

### Quitamos caracteres de fin de linea, non breaking space(\xa0), etc

In [18]:
train_origi["descripcion"] = train_origi.apply(lambda x: u' '.join(x["descripcion"].split()), axis=1)
test_origi["descripcion"] = test_origi.apply(lambda x: u' '.join(x["descripcion"].split()), axis=1)

In [19]:
train_origi.loc[349].descripcion

'vendo bonita casa en fracc sobre forjadores, caseta de vigilancia. sala, comedor, cocina con area para desayunador, bodega, patio trasero, medio baño, sala de t.v. 3 recamaras amplias,principal con vestidor y baño, baño compartido. cochera 2 autos. citas:'

In [20]:
train_origi.loc[302].descripcion

'casa con 20 años de edad en buenas condiciones,espacios amplios, iluminada, construida en 3 niveles, 5 habitaciones amplias, 3 baños completos, no cuenta con estacionamiento, cuarto de servicio, cuarto de lavado, cocina integral, mosaico en cocina y baños, pisos de loseta española, 2 tinacos, cercana a escuelas, hospitales, iglesia san juan bautista, comercios, metro bus tenayuca, universidades, avenida vallejo, colinda con d.f. terreno de 123 m2 construcción de 245 m2. se acepta credito bancario, infonavit, fovissste'

In [21]:
train_origi.loc[802].descripcion

'venta de casa **queretaro** el mirador casa de 104.5 metros de construccion en el mirador queretaro**, hermosa casa **nueva** tres recamaras 2 1/2 baños***sport club **con alberca semiolimpica**gimnasio*canchas de futboll 7**tennis** ludoteca*kids club****video vigilancia las 24 hrs del dia por internet y accesos controlados**.terreno 90 mtsplanta baja**estancia,sala de tv,baño de visitas (1/2 baño),comedor,cocina,patio de servicio,2 cajon de estacionamiento,jardin posterior.planta alta**recamara principal con baño completo vestidor c/balcon,**segunda recamaras c closet, baño completo..preparacion jacussi en roof garden**'

### Parece que hay algunos tags estilo markdown, veamos de quitar esos

In [22]:
train_origi["descripcion"] = train_origi["descripcion"].apply(lambda x: markdown_to_text(x))
test_origi["descripcion"] = test_origi["descripcion"].apply(lambda x: markdown_to_text(x))

In [23]:
train_origi.loc[802].descripcion

'venta de casa   queretaro   el mirador casa de 104.5 metros de construccion en el mirador queretaro  , hermosa casa   nueva   tres recamaras 2 1/2 baños   sport club   con alberca semiolimpica  gimnasio canchas de futboll 7  tennis   ludoteca kids club    video vigilancia las 24 hrs del dia por internet y accesos controlados  .terreno 90 mtsplanta baja  estancia,sala de tv,baño de visitas (1/2 baño),comedor,cocina,patio de servicio,2 cajon de estacionamiento,jardin posterior.planta alta  recamara principal con baño completo vestidor c/balcon,  segunda recamaras c closet, baño completo..preparacion jacussi en roof garden  '

### Quitamos acentos que hayan quedado

In [24]:
train_origi["descripcion"] = train_origi["descripcion"].apply(lambda x: strip_accents(x))
test_origi["descripcion"] = test_origi["descripcion"].apply(lambda x: strip_accents(x))

In [25]:
train_origi.to_csv(f"{ruta_data}/descripciones_train_limpias.csv", index=False)

In [26]:
test_origi.to_csv(f"{ruta_data}/descripciones_test_limpias.csv", index=False)

In [35]:
train_origi[train_origi["descripcion"]==""].shape

(1844, 1)

In [30]:
prue = pd.read_csv(f"{ruta_data}/descripciones_train_limpias.csv")

In [32]:
prue.isna().sum()

descripcion    1844
dtype: int64